In [ ]:
# check if we are running in google colab
# needed since in colab, we have to install conda and a few packages
# from conda-forge

import os
import distutils
import sys
import subprocess

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

from distutils import spawn
CUDA_PRESENT = spawn.find_executable('nvidia-smi') is not None  

if IN_COLAB:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'condacolab'])
    import condacolab
    condacolab.install()
    # the kernel gets restarted here (after conda install)
    # so all variables are deleted if you run this cell
    subprocess.run(['mamba', 'install', 'parallelproj'])
    if CUDA_PRESENT:
        subprocess.run(['mamba', 'install', 'cupy'])

# we need to redo the check for COLAB, because the install
# of conda on colab, restarts the kernel and deletes all variables
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

CUDA_PRESENT = distutils.spawn.find_executable('nvidia-smi') is not None 

if IN_COLAB:
    os.environ['PARALLELPROJ_C_LIB']='/usr/local/lib/libparallelproj_c.so'
    if CUDA_PRESENT:
        os.environ['PARALLELPROJ_CUDA_LIB']='/usr/local/lib/libparallelproj_cuda.so'

print(f'notebook running in google colab: {IN_COLAB}')
print(f'cuda present: {CUDA_PRESENT}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import parallelproj

if parallelproj.cupy_enabled:
    import cupy as cp
    import cupy as xp
else:
    import numpy as xp

In [ ]:
def tonumpy(x):
    """ function that converts a cupy or numpy array to an numpy array"""
    try:
        if cp.get_array_module(x).__name__ == 'cupy':
            return cp.asnumpy(x)
        else:
            return x
    except:
        return x

In [ ]:
# setup a test image

# image dimensions
n0, n1, n2 = (1, 128, 128)
img_dim = (n0, n1, n2)

# voxel size of the image
voxel_size = xp.array([2., 2., 2.]).astype(xp.float32)

# image origin -> world coordinates of the [0,0,0] voxel
img_origin = ((-xp.array(img_dim) / 2 + 0.5) * voxel_size).astype(xp.float32)

# setup a random image
img = xp.zeros((n0,n1,n2)).astype(xp.float32)
img[:,(n1//4):((3*n1)//4), (n2//4):((3*n2)//4)] = 1
img[:,(n1//4):(n1//3), (n2//4):(n2//3)] = 3


In [ ]:
import abc

class LinearOperator(abc.ABC):
    @property
    @abc.abstractmethod
    def ishape(self):
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def oshape(self):
        raise NotImplementedError

    @abc.abstractmethod
    def __call__(self, x):
        """ forward step y = Ax"""
        raise NotImplementedError

    @abc.abstractmethod
    def adjoint(self, y):
        """ adjoint step x = A^H y"""
        raise NotImplementedError


class ParallelViewProjector2D(LinearOperator):
    def __init__(self, image_shape, radial_positions, num_views, radius, image_origin, voxel_size):
        self._image_shape = image_shape
        self._radial_positions = radial_positions

        self._num_rad = radial_positions.shape[0]
        self._num_views = num_views

        self._radius = radius
        self._image_origin = image_origin
        self._voxel_size = voxel_size

        # array of projection angles
        self._view_angles = xp.linspace(0, xp.pi, self._num_views, endpoint=False)

        self._xstart = xp.zeros((self._num_views, self._num_rad, 3), dtype = xp.float32)
        self._xend = xp.zeros((self._num_views, self._num_rad, 3), dtype = xp.float32)

        for i, phi in enumerate(self._view_angles):
            # world coordinates of LOR start points
            self._xstart[i,:,1] = xp.cos(phi) * r + xp.sin(phi)*self._radius
            self._xstart[i,:,2] = -xp.sin(phi) * r + xp.cos(phi)*self._radius
            # world coordinates of LOR endpoints
            self._xend[i,:,1] = xp.cos(phi) * r - xp.sin(phi)*self._radius
            self._xend[i,:,2] = -xp.sin(phi) * r - xp.cos(phi)*self._radius

    @property
    def ishape(self):
        return self._image_shape

    @property
    def oshape(self):
        return (self._num_views, self._num_rad)

    def __call__(self, x):
        y = xp.zeros(self.oshape, dtype=xp.float32)
        parallelproj.joseph3d_fwd(self._xstart.reshape(-1,3), self._xend.reshape(-1,3), x, self._image_origin, self._voxel_size, y)
        return y

    def adjoint(self, y):
        x = xp.zeros(self.ishape, dtype=xp.float32)
        parallelproj.joseph3d_back(self._xstart.reshape(-1,3), self._xend.reshape(-1,3), x, self._image_origin, self._voxel_size, y)
        return x
        
    def show_views(self, views_to_show = None, image = None, **kwargs):
        if views_to_show is None:
            views_to_show = np.linspace(0, self._num_views-1, 5).astype(int)

        num_cols = len(views_to_show)
        fig, ax = plt.subplots(1, num_cols, figsize=(num_cols*3,3))
        
        tmp1 = float(self._image_origin[1] - 0.5*self._voxel_size[1])
        tmp2 = float(self._image_origin[2] - 0.5*self._voxel_size[2])
        img_extent = [tmp1,-tmp1,tmp2,-tmp2]
        
        for i, ip in enumerate(views_to_show):
            ax[i].plot(tonumpy(self._xstart[ip,:,1]),tonumpy(self._xstart[ip,:,2]),'.', ms = 0.5)
            ax[i].plot(tonumpy(self._xend[ip,:,1]),tonumpy(self._xend[ip,:,2]),'.', ms = 0.5)
            for k in np.linspace(0, num_rad-1, 7).astype(int):
                ax[i].plot([float(self._xstart[ip,k,1]), float(self._xend[ip,k,1])], [float(self._xstart[ip,k,2]), float(self._xend[ip,k,2])], 'k-', lw = 0.5)
                ax[i].annotate(f'{k}', (float(self._xstart[ip,k,1]), float(self._xstart[ip,k,2])), fontsize = 'xx-small')
            ax[i].set_xlim(-500,500)
            ax[i].set_ylim(-500,500)
            ax[i].grid(ls=':')
            ax[i].set_aspect('equal')
        
            if img is not None:
                ax[i].add_patch(Rectangle((tmp1, tmp2), float(self.ishape[1]*self._voxel_size[1]), float(self.ishape[2]*self._voxel_size[2]), edgecolor = 'r', facecolor = 'none', linestyle = ':'))
                ax[i].imshow(tonumpy(image[0,...]).T, origin = 'lower', extent = img_extent, **kwargs)
            ax[i].set_title(f'view {ip:03} - phi {(180/np.pi)*self._view_angles[ip]:.1f} deg', fontsize = 'small') 
        
        fig.tight_layout()

        return fig

In [ ]:
# setup the coordinates for projections along parallel views
num_rad = 223
num_phi = 190

# radial coordinates of the projection views in mm
r = xp.linspace(-200, 200, num_rad, dtype = xp.float32)
# "radius" of the scanner in mm
R = 350.

projector = ParallelViewProjector2D(img_dim, r, num_phi, R, img_origin, voxel_size)

fig = projector.show_views(image = img, cmap = 'Greys')



In [ ]:
# do a non-TOF forward projection
img_fwd = projector(img)

In [ ]:
# do a non-TOF back projection (the adjoint of the forward projection)
img_back = projector.adjoint(img_fwd)


In [ ]:
im_kwargs = dict(origin = 'lower', cmap = 'Greys')

fig2, ax2 = plt.subplots(1,3,figsize=(12,4))
im0 = ax2[0].imshow(tonumpy(img[0,...]).T, **im_kwargs)
im1 = ax2[1].imshow(tonumpy(img_fwd), cmap = 'Greys')
im2 = ax2[2].imshow(tonumpy(img_back[0,...]).T, **im_kwargs)
ax2[0].set_title('image - x', fontsize = 'small')
ax2[1].set_title('forward projection - Ax', fontsize = 'small')
ax2[2].set_title('back projection of forward projection - A^HAx', fontsize = 'small'
)
cb0 = fig.colorbar(im0, fraction = 0.03)
cb1 = fig.colorbar(im1, fraction = 0.03)
cb2 = fig.colorbar(im2, fraction = 0.03)

ax2[0].set_xlabel('x1')
ax2[0].set_ylabel('x2')

ax2[1].set_xlabel('radial bin')
ax2[1].set_ylabel('view number')

ax2[2].set_xlabel('x1')
ax2[2].set_ylabel('x2')

fig2.tight_layout()